In [ ]:
import os
import time
import numba
from numba import jit
from numba.core import types
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

# Util functions for the Simulated Annealing

- most of them were available/coded in the assignment docs

In [ ]:
@jit(nopython=True)
def newpath(cam, dist):
    N = len(cam)
    ncam = np.zeros(N, dtype=np.int16)

    # Seleciona i e j aleatoriamente de forma que i ≠ j
    i = np.random.randint(N)
    j = i
    while j == i:
        j = np.random.randint(N)
    
    # Define ini e fim de forma que ini < fim
    if i > j:
        ini = j
        fim = i
    else:
        ini = i
        fim = j

    # Cria o novo caminho ncam baseado em cam
    for k in range(N):
        if ini <= k <= fim:
            ncam[k] = cam[fim - (k - ini)]
        else:
            ncam[k] = cam[k]

    # Calcula as diferenças nas extremidades do intervalo
    esq = ini - 1
    if esq < 0:
        esq = N - 1
    dir = fim + 1
    if dir > N - 1:
        dir = 0

    de = (-dist[cam[esq], cam[ini]] - dist[cam[dir], cam[fim]] 
          + dist[ncam[esq], ncam[ini]] + dist[ncam[dir], ncam[fim]])

    return ncam, de

#define a distancia entre duas cidades quaisquer
@jit(nopython=True)
def distances(N,x,y):
    
    dist = np.zeros((N,N),dtype=np.float32)
    for i in range(N):
        for j in range(N):
            dist[i,j] = np.sqrt((x[i]-x[j])*(x[i]-x[j])+(y[i]-y[j])*(y[i]-y[j]))
            
    return dist

@jit(nopython=True)
def cost(N,path,dist):
    # calcula a distancia total percorrida pela caminhada
    ener = 0
    for i in range(N-1):
        ener += dist[path[i],path[i+1]]
    ener += dist[path[0],path[N-1]]     # conecta a última e a primeira cidades do caminho
    
    return ener

# Plot functions

In [ ]:
def plot_tsp(X, Y, path, cost, temp, save=False, save_path='graphs', filename=None):
    plt.figure(figsize=(8, 6))
    
    plt.scatter(X, Y, color='blue')
    
    for i in range(len(path)):
        start = path[i]
        end = path[(i + 1) % len(path)]
        plt.plot([X[start], X[end]], [Y[start], Y[end]], 'r-')
    
    # Annotate the points
    for i, (x, y) in enumerate(zip(X, Y)):
        plt.text(x, y, f'{i}', fontsize=10, ha='right')
    
    plt.title(f'TSP path - cost: {cost} - temperature: {temp}')
    plt.xlabel('X Coordinates')
    plt.ylabel('Y Coordinates')
    plt.grid(True)
    
    if save:
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        if filename is None:
            filename = f'tsp_path_cost_{np.round(cost, 5)}_temp_{temp}.png'
        
        plt.savefig(os.path.join(save_path, filename))
        plt.close()
        
    else:
        plt.show()
    

def plot_tsp_side_by_side(X, Y, method_name1, path1, cost1, method_name2, path2, cost2):
    fig, axs = plt.subplots(1, 2, figsize=(16, 8))

    # First plot
    axs[0].scatter(X, Y, color='blue')
    for i in range(len(path1)):
        start = path1[i]
        end = path1[(i + 1) % len(path1)]
        axs[0].plot([X[start], X[end]], [Y[start], Y[end]], 'r-')
    for i, (x, y) in enumerate(zip(X, Y)):
        axs[0].text(x, y, f'{i}', fontsize=12, ha='right')
    axs[0].set_title(f'{method_name1} - cost: {np.round(cost1, 4)}')
    axs[0].set_xlabel('X Coordinates')
    axs[0].set_ylabel('Y Coordinates')
    axs[0].grid(True)

    # Second plot
    axs[1].scatter(X, Y, color='blue')
    for i in range(len(path2)):
        start = path2[i]
        end = path2[(i + 1) % len(path2)]
        axs[1].plot([X[start], X[end]], [Y[start], Y[end]], 'r-')
    for i, (x, y) in enumerate(zip(X, Y)):
        axs[1].text(x, y, f'{i}', fontsize=12, ha='right')
    axs[1].set_title(f'{method_name2} - cost: {np.round(cost2, 4)}')
    axs[1].set_xlabel('X Coordinates')
    axs[1].set_ylabel('Y Coordinates')
    axs[1].grid(True)

    plt.show()

# Simulated Annealing main function

In [ ]:
@jit(nopython=True)
def simulated_annealing(
    config: numba.typed.Dict,
    N: int,
    x: np.ndarray,
    y: np.ndarray,
    inipath: np.ndarray,
    plot=False) -> tuple[np.array, float]:
    """
    Executes the simulated annealing algorithm in a 2d space to solve TSP.
    
    Args:
        config (numba.typed.Dict):
            This config dict must be numba integrated, that's why the type
            it must have the keys:
            - 'T': initial temperature,
            - 'dT': decaying temperature factor,
            - 'Tf': final minimum temperature,
            - 'steps': steps for monte carlo in the same temperature,
            - 'iterations': number of iterations of monte carlos batches,
        
        N (int): number of cities
        x (np.ndarray): x values array
        y (np.ndarray): y values array
        inipath (np.ndarray): initial path
        
    Returns:
        (best_path, best_cost): best path and cost encountered during the execution
    """
    
    # Initialization
    distance_matrix = distances(N, x, y)
    temp = config['T']
    
    current_path = inipath.copy()
    current_cost = cost(N, current_path, distance_matrix)
    
    best_path = current_path
    best_cost = np.inf
    
    # Repeat process
    for _ in range(config['iterations']):
        # Monte Carlo Simmulations
        for _ in range(config['steps']):
            new_path, de = newpath(current_path, distance_matrix)
            
            if de < 0:
                current_path = new_path
                current_cost = cost(N, current_path, distance_matrix)
            else:
                if np.random.rand() <= np.exp(-de / temp):
                    current_path = new_path
                    current_cost = cost(N, current_path, distance_matrix)
                    
            if current_cost < best_cost:
                best_cost = current_cost
                best_path = current_path.copy()
                    
        temp = max(temp * config['dT'], config['Tf'])
    
    return best_path, best_cost

# Nearest Neighbors solution for TSP for comparation

In [ ]:
@jit(nopython=True)
def distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

@jit(nopython=True)
def nearest_neighbor(x, y, start_city):
    N = len(x)
    visited = np.zeros(N, dtype=np.int32)
    path = np.zeros(N, dtype=np.int32)
    current_city = start_city
    path[0] = current_city
    visited[current_city] = 1
    total_cost = 0.0

    for i in range(1, N):
        nearest_city = -1
        nearest_distance = np.inf
        for j in range(N):
            if not visited[j]:
                dist = distance(x[current_city], y[current_city], x[j], y[j])
                if dist < nearest_distance:
                    nearest_distance = dist
                    nearest_city = j
        path[i] = nearest_city
        visited[nearest_city] = 1
        total_cost += nearest_distance
        current_city = nearest_city
    
    # Return to the start city to complete the cycle
    total_cost += distance(x[current_city], y[current_city], x[start_city], y[start_city])
    path = np.append(path, start_city)
    
    return path, total_cost

# Configuration

Now it is necessary to specify the parameters to execute the algorithm

In [ ]:
rng = np.random.default_rng(seed=42)

# Number of cities
N = 150

x=rng.random(N)
y=rng.random(N)

# initial path
inipath = np.zeros(N,dtype=np.int16)
for i in range(N):
    inipath[i]=i

In [ ]:
config = {
    'T': 10,
    'dT': 0.95,
    'Tf': 0.001,
    'steps': 10000,
    'iterations': 1000,
}

numba_config = numba.typed.Dict.empty(
    key_type=types.unicode_type,
    value_type=types.float64
)

for key, value in config.items():
    numba_config[key] = float(value)

print(numba_config)

In [ ]:
init_sim = time.perf_counter()
best_path, best_cost = simulated_annealing(numba_config, N, x, y, inipath)
print(f'Runtime Simulated Annealing: {time.perf_counter() - init_sim}')

init_nn = time.perf_counter()
nn_path, nn_cost = nearest_neighbor(x, y, 0)
print(f'Runtime Nearest Neighbor: {time.perf_counter() - init_nn}')

In [ ]:
if N <= 80:
    plot_tsp_side_by_side(x, y, 'simulated annealing', best_path, best_cost, 'Nearest neighbor', nn_path, nn_cost)
else:
    print(f'Cost Simulated Annealing: {best_cost} | Cost Nearest Neighbor: {nn_cost}')

# Generating the video of the algorithm evolution

- It is commented because it takes a while to run

In [ ]:
def simulated_annealing_plot(
    config: dict,
    N: int,
    x: np.ndarray,
    y: np.ndarray,
    inipath: np.ndarray) -> tuple[np.array, float]:
    """
    Executes the simulated annealing algorithm in a 2d space to solve TSP.
    
    Args:
        config (dict):
            it must have the keys:
            - 'T': initial temperature,
            - 'dT': decaying temperature factor,
            - 'Tf': final minimum temperature,
            - 'steps': steps for monte carlo in the same temperature,
            - 'iterations': number of iterations of monte carlos batches,
        
        N (int): number of cities
        x (np.ndarray): x values array
        y (np.ndarray): y values array
        inipath (np.ndarray): initial path
        plot: plot the progress
        
    Returns:
        (best_path, best_cost): best path and cost encountered during the execution
    """
    
    # Initialization
    distance_matrix = distances(N, x, y)
    temp = config['T']
    
    current_path = inipath.copy()
    current_cost = -1
    
    best_path = current_path
    best_cost = np.inf
    
    # Repeat process
    for _ in range(config['iterations']):
        # Monte Carlo Simmulations
        for i in range(config['steps']):
            new_path, de = newpath(current_path, distance_matrix)
            
            if de < 0:
                current_path = new_path
                current_cost = cost(N, current_path, distance_matrix)
            else:
                if np.random.rand() <= np.exp(-de / temp):
                    current_path = new_path
                    current_cost = cost(N, current_path, distance_matrix)
                    
            if current_cost < best_cost:
                best_cost = current_cost
                best_path = current_path.copy()
            
            if i % 2 == 0:
                plot_tsp(x, y, current_path, current_cost, temp, save=True, filename=f'tsp_graph_i_{i}_t_{np.round(temp, 5)}.png')
                    
        temp = max(temp * config['dT'], config['Tf'])
    
    return best_path, best_cost

In [ ]:
# config['iterations'] = 75
# config['steps'] = 75

# simulated_annealing_plot(config, N, x, y, inipath)

In [ ]:
# import re
# import cv2

# def extract_temp_and_iter(filename):
#     match = re.search(r"tsp_graph_i_(\d+)_t_(\d+\.\d+).png", filename)
#     if match:
#         i = int(match.group(1))
#         temp = float(match.group(2))
#         return i, temp
#     return None, None

# # Path to the directory containing the images
# image_dir = 'graphs'
# images = []

# # Read all images and their temperature and iteration values
# for file in os.listdir(image_dir):
#     if file.endswith(".png"):
#         i, temp = extract_temp_and_iter(file)
#         if i is not None and temp is not None:
#             images.append((i, temp, os.path.join(image_dir, file)))

# # Sort images first by temperature (descending) and then by iteration (ascending)
# images.sort(key=lambda x: (-x[1], x[0]))

# # Parameters for the video
# output_video = 'simulated_annealing_evolution.mp4'
# frame_rate = 2  # frames per second

# # Read the first image to get the frame size
# first_image = cv2.imread(images[0][2])
# height, width, layers = first_image.shape

# # Define the codec and create VideoWriter object
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# video = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))

# # Add images to the video
# for i, temp, img_path in images:
#     img = cv2.imread(img_path)
#     video.write(img)

# # Release the video writer object
# video.release()
# cv2.destroyAllWindows()

# print(f"Video saved as {output_video}")

# Parametrization

- Now I will execute different configs and check what happens
- N = 150

In [ ]:
def parametrization_test():
    Ts = [1, 2, 4, 6, 8, 10, 12]
    dTs = [0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99]
    Tfs = [0.01, 0.001, 0.005, 0.0001]
    num_exec_per_config = 10

    results = []

    for T in Ts:
        for dT in dTs:
            for Tf in Tfs:
                costs = []
                for i in range(num_exec_per_config):
                    config = {
                        'T': T,
                        'dT': dT,
                        'Tf': Tf,
                        'steps': 1000,
                        'iterations': 1000,
                    }

                    numba_config = numba.typed.Dict.empty(
                        key_type=types.unicode_type,
                        value_type=types.float64
                    )

                    for key, value in config.items():
                        numba_config[key] = float(value)

                    _, best_cost = simulated_annealing(numba_config, N, x, y, inipath)
                    
                    costs.append(best_cost)
                    
                results.append({
                    'T': T,
                    'dT': dT,
                    'Tf': Tf,
                    'avg_cost': np.mean(costs)
                })

    df_results = pd.DataFrame(results)
    return df_results

In [ ]:
def heatmap(df_results):
    def create_heatmap(data, index, columns, values, ax, title):
        heatmap_data = data.pivot_table(index=index, columns=columns, values=values)
        sns.heatmap(heatmap_data, annot=True, cmap='viridis', ax=ax)
        ax.set_title(title)
        ax.set_xlabel(columns)
        ax.set_ylabel(index)

    # Criar a figura e os eixos para os subplots
    fig, axs = plt.subplots(1, 3, figsize=(24, 8))

    # Heatmap para T e dT
    create_heatmap(df_results, 'T', 'dT', 'avg_cost', axs[0], 'Impacto Conjunto de T e dT no Custo Médio')

    # Heatmap para Tf e T
    create_heatmap(df_results, 'T', 'Tf', 'avg_cost', axs[1], 'Impacto Conjunto de T e Tf no Custo Médio')

    # Heatmap para dT e Tf
    create_heatmap(df_results, 'dT', 'Tf', 'avg_cost', axs[2], 'Impacto Conjunto de dT e Tf no Custo Médio')

    plt.tight_layout()

    output_dir = 'images'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_path = os.path.join(output_dir, 'heatmap.png')
    plt.savefig(output_path)

    plt.show()

# Heatmap Analysis

![Heatmap](images/heatmap.png)

Diferentemente do que eu esperava, após os testes, que foram refeitos várias vezes para confirmação, obteve-se esse padrão, em que **variações menores na temperatura e uma temperatura final maior geram um menor custo médio**. Além disso, supreendentemente, **a temperatura inicial não parece interferir muito.** A minha teoria para a temperatura não influenciar tanto é que um valor baixo de T, como por exemplo 1, já proporciona uma alta probabilidade de aceitação através da expressão `np.exp(- de / temp)`. Para fins demonstrativos, com T=1 e de=0.1 (o que ocorre bastante pelos meus experimentos), a probabilidade de aceitação é **90%**.


In [ ]:
def boxplot_temp(df_results):
    sns.set_theme(style="whitegrid")

    plt.figure(figsize=(12, 8))
    sns.boxplot(data=df_results, x='T', y='avg_cost')

    plt.title('Boxplot da Temperatura (T) com Relação ao Custo Médio', fontsize=14)
    plt.xlabel('Temperatura (T)', fontsize=12)
    plt.ylabel('Custo Médio', fontsize=12)

    plt.grid(True, linestyle='--', alpha=0.6)

    plt.savefig('images/boxplot_temp.png')

    plt.show()

Para confirmar a teoria de que a temperatura não interferiu bastante, realizei um novo plot apresentando a temperatura pelo custo

![Temperature boxplot](images/boxplot_temp.png)

Verifica-se portanto, que não há diferenças significativas entre as diferentes temperaturas com relação ao resultado macroscópico do custo nesse teste

# Análise

## Valor de N e eficiência

O algoritmo de Simulated Annealing conseguiu obter um resultado melhor para quase todos valores de N experimentados do que o modelo de Nearest Neighbor em um tempo relativamente semelhante (aos olhos humanos). 

Quando digo semelhante, digo que para nos humanos esperarmos 0.5 segundos ou 2 segundos não faz tanta diferença, mas houve uma diferença absoluta considerável: em todos os casos independente do tamanho que testei, que chegou em N = 8000, o algoritmo de Nearest Neighbor sempre executou em menos de 0.5 segundo, enquanto o algoritmo de Simulated Annealing executava com tempos variados dependendo da parametrização, mas mantendo sempre um tempo de execução maior do que 2 segundos com resultados bons.

A partir de N >= 5000, começou-se a notar um tempo significativamente maior para o Simulated Annealing e resultados não tão bons com relação ao Nearest Neighbor. O que levou em uma reparametrização do modelo e uma consequentemente melhoria nos resultados acompanhado de um aumento no tempo de execução.

**Para N = 5000, um dos experimentos resultou em:**

---

`
config = {
    'T': 10,
    'dT': 0.95,
    'Tf': 0.0005,
    'steps': 10000,
    'iterations': 10000 }
`

Runtime Simulated Annealing: **287.158368847 s**
Runtime Nearest Neighbor: **0.025974851000228227 s**

Cost Simulated Annealing: **57.730015** | Cost Nearest Neighbor: **63.511542**

---

Através desse experimento específico, verifica-se uma característica do simulated annealing: é possível obter um resultado arbitrariamente bom à medida em que se aumenta o número de iterações.


## Parametrização

Como discutido anteriormente, a temperatura inicial aparentemente não é um fator determinante no algoritmo. Para uma análise mais aprofundada, seria necessário realizar mais testes demorados, que ficarão para um futuro trabalho. No entanto, foi identificado a influência dos parâmetros dT e Tf claramente nos testes. Possivelmente a organização randomica obtida influenciou nos resultados, mas não sei mensurar quanto.

Além disso, é claro que o número de passos e iterações do modelo afetam o custo: quanto maior esses parâmetros, o algoritmo tende a achar menores custos.


# Vídeo de evolução do algoritmo

<div style="padding: 20px 0;">
  <video width="560" height="315" controls>
    <source src="videos/evolution_fast.mp4" type="video/mp4">
    Seu navegador não suporta a tag de vídeo.
  </video>
</div>

- Algo interessante: no último passo do algoritmo ele efetuou um novo grande caminho

[Vídeo](videos/evolution_fast.mp4)


[Repositorio no GitHub](https://github.com/lorenzo-cm/simulated-annealing)